# Resumen

Hasta ahora hemos implementado distintos modelos de estimación o regresión:

* Regresión lineal, de una variable
* Regresión de varias variables
* Regresión polinómica, incluyendo transformación de atributos e interacciones entre ellos.

Para medir el error hemos utilizado el error cuadrático medio sobre **todas** las muestras de entrenamiento. En este Notebook, vamos a analizar las prestaciones de cuatro modelos de regresión, pero ya haciendolo bien, es decir con sus dataset de test y entrenamiento.

* Modelo 1: utilizaremos el atributo *sqm_living* de manera aislada.
* Modelo 2: utilizaremos las siguientes variables
    * *price*, *sqm_living*, *bedrooms*, *bathrooms*, *floors*, *years*, *zip_code*
* Modelo 3: utilizaremos las siguientes variables
    * *price*, *sqm_living*, *bedrooms_squared*, *bed_bath_rooms*, *log_sqm_living*, *lat_plus_long*, *sqm_living*
 

        * Elevar al cuadrado el número de habitaciones incrementará la separación entre viviendas de no muchas habitaciones,  y las viviendas con muchas habitaciones 
        * Multiplicar las variables bedrooms y bathrooms genera un atributo cuyo valor será grande cuando conjuntamente ambas variables lo sean.
        * Utilizar el logaritmo del tamaño de la vivienda separa lo valores pequeños y agrupa los valores grandes.
        * Sumar latitud y longitud no mucho tiene sentido, pero la añadimos igualmente para ver que supone no conocer el negocio
* Modelo 4: utilizaremos las variables originales de la base de datos

In [20]:
# cargamos librerías necesarias
import numpy  as np  
import pandas as pd

import matplotlib.pyplot as plt # para dibujar
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from math import log

## Cargar datos

In [8]:
# cargamos fichero
house_data = pd.read_csv("./data/house_data.csv") 

# Eliminamos las columnas id y date 
house_data = house_data.drop(['id','date'], axis=1)

# convertir las variables en pies al cuadrado en metros al cuadrado 
feetFeatures = ['sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15']
house_data[feetFeatures] = house_data[feetFeatures].apply(lambda x: x * 0.3048 * 0.3048)

# renombramos
house_data.columns = ['price','bedrooms','bathrooms','sqm_living','sqm_lot','floors','waterfront','view','condition',
                      'grade','sqm_above','sqm_basement','yr_built','yr_renovated','zip_code','lat','long',
                      'sqm_living15','sqm_lot15']

# añadimos las nuevas variables
house_data['years']            = 2018 - house_data['yr_built']
house_data['bedrooms_squared'] = house_data['bedrooms'].apply(lambda x: x**2)
house_data['bed_bath_rooms']   = house_data['bedrooms']*house_data['bathrooms']
house_data['log_sqm_living']   = house_data['sqm_living'].apply(lambda x: log(x))
house_data['lat_plus_long']    = house_data['lat']*house_data['long']

In [9]:
house_data.head(5)

,price,bedrooms,bathrooms,sqm_living,sqm_lot,floors,waterfront,view,condition,grade,...,zip_code,lat,long,sqm_living15,sqm_lot15,years,bedrooms_squared,bed_bath_rooms,log_sqm_living,lat_plus_long
0,221900.0,3,1.00,109.625587,524.902176,1.0,0,0,3,7,...,98178,47.5112,-122.257,124.490074,524.902176,62,9,3.00,4.697071,-5808.576778
1,538000.0,3,2.25,238.760813,672.803816,2.0,0,0,3,7,...,98125,47.7210,-122.319,157.006138,709.686323,66,9,6.75,5.475462,-5837.184999
2,180000.0,2,1.00,71.535341,929.030400,1.0,0,0,3,6,...,98028,47.7379,-122.233,252.696269,748.984308,84,4,2.00,4.270192,-5835.146731
3,604000.0,4,3.00,182.089958,464.515200,1.0,0,0,5,7,...,98136,47.5208,-122.393,126.348134,464.515200,52,16,12.00,5.204501,-5816.213274
4,510000.0,3,2.00,156.077107,750.656563,1.0,0,0,3,8,...,98074,47.6168,-122.045,167.225472,697.051509,30,9,6.00,5.050350,-5811.392356


## Entrenamiento/Test

Dividamos los datos en un conjunto de entrenamiento y test

In [11]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
print(len(house_data))
house_data =house_data.dropna()
print(len(house_data))
# convertimos el DataFrame al formato necesario para scikit-learn
data = house_data.as_matrix() 

y = data[:,0:1]     # nos quedamos con la 1ª columna, price
X = data[:,1:]     # nos quedamos con el resto (sqm_living)

feature_names = house_data.columns[1:]

# Dividimos los datos en entrenamiento y test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=.25, random_state = 2)

print ('Datos entrenamiento: ', Xtrain.shape)
print ('Datos test: ', Xtest.shape)

# Escalamos (con los datos de train)
scaler = preprocessing.StandardScaler().fit(Xtrain)
XtrainScaled = scaler.transform(Xtrain)
XtestScaled = scaler.transform(Xtest)

3165
3164
Datos entrenamiento:  (2373, 23)
Datos test:  (791, 23)


<div class = "alert alert-success">
¿Qué porcentaje de las muestras totales utilizamos para construir nuestro conjunto de test?
</div>

In [13]:
# Modelo 1
XtrainModel1 = XtrainScaled[:,2:3]
XtestModel1 = XtestScaled[:,2:3]

Model1 = LinearRegression()
Model1.fit(XtrainModel1,ytrain)

ytrain1 = Model1.predict(XtrainModel1)
ytest1 = Model1.predict(XtestModel1)
mseTrainModel1 = mean_squared_error(ytrain,ytrain1)
mseTestModel1 = mean_squared_error(ytest,ytest1)

print ('MSE Modelo 1 (train): %0.3g' % mseTrainModel1)
print ('MSE Modelo 1 (test) : %0.3g' % mseTestModel1)

MSE Modelo 1 (train): 7.29e+10
MSE Modelo 1 (test) : 7.31e+10


In [14]:
print (feature_names)

Index(['bedrooms', 'bathrooms', 'sqm_living', 'sqm_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqm_above', 'sqm_basement',
       'yr_built', 'yr_renovated', 'zip_code', 'lat', 'long', 'sqm_living15',
       'sqm_lot15', 'years', 'bedrooms_squared', 'bed_bath_rooms',
       'log_sqm_living', 'lat_plus_long'],
      dtype='object')


In [16]:
# Modelo 2
featuresModel2 = [0,1,2,4,13,18]
#print 'Modelo entrenado con: ', feature_names[featuresModel2]

XtrainModel2 = XtrainScaled[:,featuresModel2]
XtestModel2 = XtestScaled[:,featuresModel2]

Model2 = LinearRegression()
Model2.fit(XtrainModel2,ytrain)

ytrain2 = Model2.predict(XtrainModel2)
ytest2  = Model2.predict(XtestModel2)
mseTrainModel2 = mean_squared_error(ytrain,ytrain2)
mseTestModel2 = mean_squared_error(ytest,ytest2)

print ('MSE Modelo 2 (train): %0.3g' % mseTrainModel2)
print ('MSE Modelo 2 (test) : %0.3g' % mseTestModel2)

MSE Modelo 2 (train): 6.34e+10
MSE Modelo 2 (test) : 6.48e+10


In [17]:
# Modelo 3: price, sqm_living, bedrooms_squared, bed_bath_rooms, log_sqm_living, lat_plus_long, sqm_living
featuresModel3 = [2,18,19,20,21,22]
#print 'Modelo entrenado con: ', feature_names[featuresModel3]

XtrainModel3 = XtrainScaled[:,featuresModel3]
XtestModel3  = XtestScaled[:,featuresModel3]

Model3 = LinearRegression()
Model3.fit(XtrainModel3,ytrain)

ytrain3 = Model3.predict(XtrainModel3)
ytest3  = Model3.predict(XtestModel3)
mseTrainModel3 = mean_squared_error(ytrain,ytrain3)
mseTestModel3 = mean_squared_error(ytest,ytest3)

print ('MSE Modelo 3 (train): %0.3g' % mseTrainModel3)
print ('MSE Modelo 3 (test) : %0.3g' % mseTestModel3)

MSE Modelo 3 (train): 5.36e+10
MSE Modelo 3 (test) : 5.47e+10


In [19]:
# Modelo 4: todas las características originales
print ('Modelo entrenado con: ', feature_names[0:18])

XtrainModel4 = XtrainScaled[:,0:18]
XtestModel4  = XtestScaled[:,0:18]

Model4 = LinearRegression()
Model4.fit(XtrainModel4,ytrain)

ytrain4 = Model4.predict(XtrainModel4)
ytest4  = Model4.predict(XtestModel4)
mseTrainModel4 = mean_squared_error(ytrain,ytrain4)
mseTestModel4 = mean_squared_error(ytest,ytest4)

print ('MSE Modelo 3 (train): %0.3g' % mseTrainModel4)
print ('MSE Modelo 3 (test) : %0.3g' % mseTestModel4)

Modelo entrenado con:  Index(['bedrooms', 'bathrooms', 'sqm_living', 'sqm_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqm_above', 'sqm_basement',
       'yr_built', 'yr_renovated', 'zip_code', 'lat', 'long', 'sqm_living15',
       'sqm_lot15'],
      dtype='object')
MSE Modelo 3 (train): 4.02e+10
MSE Modelo 3 (test) : 4.28e+10


### Ejercicio

<div class = "alert alert-success">
¿Qué modelo tiene menor error de entrenamiento? ¿Y cuál tiene menor error en test?
</div>